In [1]:
import pandas as pd
import numpy as np

In [2]:
data_train = pd.read_csv('./data/application_train.csv')
data_test = pd.read_csv('./data/application_test.csv')

In [3]:
data_train_1 = data_train.loc[data_train.loc[data_train.index,'TARGET']==1]
data_train_0 = data_train.loc[data_train.loc[data_train.index,'TARGET']==0]

data_train_1.shape, data_train_0.shape

((24825, 122), (282686, 122))

In [4]:
data_train_0 = data_train_0[:len(data_train_1)]

In [5]:
data_train = pd.concat((data_train_0,data_train_1)).sort_index()
data_train.shape, data_test.shape

((49650, 122), (48744, 121))

In [6]:
data_train = data_train.set_index(data_train.SK_ID_CURR)
data_test = data_test.set_index(data_test.SK_ID_CURR)

data_train = data_train.drop('SK_ID_CURR',axis=1)
data_test = data_test.drop('SK_ID_CURR',axis=1)

In [7]:
%run ./code/label_encoder.py

In [8]:
#se codifican los datos tanto de entrenamiento y prueba
data_train = label_encoder(data_train)
data_test = label_encoder(data_test)

In [9]:
#saco la media de las columnas que presentan valores NaN
col_mean = (data_train.loc[:,data_train.count()<data_train.shape[0]].min())
dict_col_mean = {}
for i in range(len(col_mean)):
    dict_col_mean[col_mean.index[i]] = col_mean[i]

In [10]:
#lo mismo para el data_test
col_mean_test = (data_test.loc[:,data_test.count()<data_test.shape[0]].min())
dict_col_mean_test = {}
for i in range(len(col_mean_test)):
    dict_col_mean_test[col_mean_test.index[i]] = col_mean_test[i]

In [11]:
#se filtra con el diccionario obtenido anteriormente
data_train = data_train.fillna(value=dict_col_mean)
data_test = data_test.fillna(value=dict_col_mean_test)

In [12]:
y_train = data_train.TARGET

In [13]:
X_train = data_train[[col for col in  data_train.columns if col!='TARGET']]
X_test = data_test[[col for col in  data_test.columns if col!='TARGET']]

X_train.shape, y_train.shape, X_test.shape

((49650, 120), (49650,), (48744, 120))

In [14]:
pre_app = pd.read_csv('./data/previous_application.csv')

In [15]:
pre_app = pre_app.set_index(pre_app.SK_ID_CURR.values)
pre_app = pre_app.drop('SK_ID_CURR',axis=1)

In [16]:
pre_app = pre_app.groupby(pre_app.index).mean()

In [17]:
pre_app = pre_app.drop('SK_ID_PREV',axis=1)

In [18]:
#como se ve, algunos clientes no tienen aplicaciones previas, entonces se saca la media de los otros para agregarlos
pre_app.filter(X_train.index, axis=0).shape

(47353, 19)

In [18]:
#saco la media de las columnas que presentan valores NaN
col_mean = (pre_app.loc[:,pre_app.count()<pre_app.shape[0]].min())
dict_col_mean = {}
for i in range(len(col_mean)):
    dict_col_mean[col_mean.index[i]] = col_mean[i]

In [19]:
pre_app = pre_app.fillna(value=dict_col_mean)

In [20]:
X_train_merge = pd.merge(X_train, pre_app, left_index=True, right_index=True, how='left')
X_test_merge = pd.merge(X_test, pre_app, left_index=True, right_index=True, how='left')

X_train_merge.shape, X_test_merge.shape

((49650, 139), (48744, 139))

In [21]:
#lo mismo para el data_train
col_mean_train = (X_train_merge.loc[:,X_train_merge.count()<X_train_merge.shape[0]].min())
dict_col_mean_train = {}
for i in range(len(col_mean_train)):
    dict_col_mean_train[col_mean_train.index[i]] = col_mean_train[i]
    
#lo mismo para el data_test
col_mean_test = (X_test_merge.loc[:,X_test_merge.count()<X_test_merge.shape[0]].min())
dict_col_mean_test = {}
for i in range(len(col_mean_test)):
    dict_col_mean_test[col_mean_test.index[i]] = col_mean_test[i]

In [22]:
X_train_merge = X_train_merge.fillna(value=dict_col_mean_train)
X_test_merge = X_test_merge.fillna(value=dict_col_mean_test)

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier()
GBC.fit(X_train_merge, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [26]:
GBC.score(X_train_merge, y_train)

0.70316213494461233

In [27]:
predict = GBC.predict_proba(X_test_merge)

In [28]:
predict[:,1]

array([ 0.32922225,  0.50170019,  0.2059312 , ...,  0.39140166,
        0.4688349 ,  0.74790013])

### POR AHORA EL MEJOR RESULTADO OBTENIDO

In [29]:
#sin correlación de datos
result = pd.DataFrame(data=predict[:,1], index=X_test_merge.index, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_29.csv')

### Nueva Prueba
Ahora trataŕe de que el dataset no esté balanceado 50/50, si no que haya un poco mas de las etiquetas de zero

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_train = pd.read_csv('./data/application_train.csv')
data_test = pd.read_csv('./data/application_test.csv')

In [3]:
data_train_1 = data_train.loc[data_train.loc[data_train.index,'TARGET']==1]
data_train_0 = data_train.loc[data_train.loc[data_train.index,'TARGET']==0]

data_train_1.shape, data_train_0.shape

((24825, 122), (282686, 122))

In [4]:
data_train_0 = data_train_0[:(len(data_train_1)+int(.2*len(data_train_1)))]

In [5]:
data_train = pd.concat((data_train_0,data_train_1)).sort_index()
data_train.shape, data_test.shape

((54615, 122), (48744, 121))

In [6]:
data_train = data_train.set_index(data_train.SK_ID_CURR)
data_test = data_test.set_index(data_test.SK_ID_CURR)

data_train = data_train.drop('SK_ID_CURR',axis=1)
data_test = data_test.drop('SK_ID_CURR',axis=1)

In [7]:
%run ./code/label_encoder.py

In [8]:
#se codifican los datos tanto de entrenamiento y prueba
data_train = label_encoder(data_train)
data_test = label_encoder(data_test)

In [9]:
#saco la media de las columnas que presentan valores NaN
col_mean = (data_train.loc[:,data_train.count()<data_train.shape[0]].min())
dict_col_mean = {}
for i in range(len(col_mean)):
    dict_col_mean[col_mean.index[i]] = col_mean[i]

In [10]:
#lo mismo para el data_test
col_mean_test = (data_test.loc[:,data_test.count()<data_test.shape[0]].min())
dict_col_mean_test = {}
for i in range(len(col_mean_test)):
    dict_col_mean_test[col_mean_test.index[i]] = col_mean_test[i]

In [11]:
#se filtra con el diccionario obtenido anteriormente
data_train = data_train.fillna(value=dict_col_mean)
data_test = data_test.fillna(value=dict_col_mean_test)

In [12]:
y_train = data_train.TARGET

In [13]:
X_train = data_train[[col for col in  data_train.columns if col!='TARGET']]
X_test = data_test[[col for col in  data_test.columns if col!='TARGET']]

X_train.shape, y_train.shape, X_test.shape

((54615, 120), (54615,), (48744, 120))

....

In [14]:
pre_app = pd.read_csv('./data/previous_application.csv')

In [15]:
pre_app = pre_app.set_index(pre_app.SK_ID_CURR.values)
pre_app = pre_app.drop('SK_ID_CURR',axis=1)

In [16]:
pre_app = pre_app.groupby(pre_app.index).mean()

In [17]:
pre_app = pre_app.drop('SK_ID_PREV',axis=1)

In [18]:
#saco la media de las columnas que presentan valores NaN
col_mean = (pre_app.loc[:,pre_app.count()<pre_app.shape[0]].min())
dict_col_mean = {}
for i in range(len(col_mean)):
    dict_col_mean[col_mean.index[i]] = col_mean[i]

In [19]:
pre_app = pre_app.fillna(value=dict_col_mean)

In [20]:
X_train_merge = pd.merge(X_train, pre_app, left_index=True, right_index=True, how='left')
X_test_merge = pd.merge(X_test, pre_app, left_index=True, right_index=True, how='left')

X_train_merge.shape, X_test_merge.shape

((54615, 139), (48744, 139))

In [21]:
#lo mismo para el data_train
col_mean_train = (X_train_merge.loc[:,X_train_merge.count()<X_train_merge.shape[0]].min())
dict_col_mean_train = {}
for i in range(len(col_mean_train)):
    dict_col_mean_train[col_mean_train.index[i]] = col_mean_train[i]
    
#lo mismo para el data_test
col_mean_test = (X_test_merge.loc[:,X_test_merge.count()<X_test_merge.shape[0]].min())
dict_col_mean_test = {}
for i in range(len(col_mean_test)):
    dict_col_mean_test[col_mean_test.index[i]] = col_mean_test[i]

In [22]:
X_train_merge = X_train_merge.fillna(value=dict_col_mean_train)
X_test_merge = X_test_merge.fillna(value=dict_col_mean_test)

In [23]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier()
GBC.fit(X_train_merge, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [24]:
GBC.score(X_train_merge, y_train)

0.70502609173304043

In [25]:
predict = GBC.predict_proba(X_test_merge)[:,1]

In [26]:
predict

array([ 0.29764798,  0.49639124,  0.17980009, ...,  0.33208665,
        0.39260308,  0.74258643])

In [30]:
#sin correlación de datos
result = pd.DataFrame(data=predict, index=X_test_merge.index, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_31.csv')

### Conclusión del PCA

No me arroja buenos resultados....

### Uniré la tabla de bureau para ver que tal va...

In [61]:
bureau = pd.read_csv('./data/bureau.csv')
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [36]:
bureau.shape

(1716428, 17)

In [62]:
bureau = bureau.set_index(bureau.columns[0])

In [63]:
bureau = bureau.drop('SK_ID_BUREAU', axis=1)
bureau.head()

,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
SK_ID_CURR,,,,,,,,,,,,,,,
215354,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
215354,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
215354,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
215354,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
215354,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [64]:
X_train_merge.shape, bureau.shape

((54615, 139), (1716428, 15))

In [65]:
X_train_merge.filter(bureau.index, axis=0).shape

(256344, 139)

In [66]:
bureau = label_encoder(bureau)

In [67]:
#como se repiten los datos de un cliente, sacaré la media
bureau = bureau.groupby(bureau.index).mean()
bureau.shape

(305811, 15)

In [68]:
X_train_merge.filter(bureau.index, axis=0).shape, X_test_merge.filter(bureau.index, axis=0).shape 

((45983, 139), (42320, 139))

In [69]:
X_train_merge.shape, X_test_merge.shape

((54615, 139), (48744, 139))

In [70]:
bureau.head()

,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
SK_ID_CURR,,,,,,,,,,,,,,,
100001,1.142857,0.0,-735.000000,0.0,82.428571,-825.500000,NaN,0.0,207623.571429,85240.928571,0.00000,0.0,3.000000,-93.142857,3545.357143
100002,1.500000,0.0,-874.000000,0.0,-349.000000,-697.500000,1681.029,0.0,108131.945625,49156.200000,7997.14125,0.0,3.500000,-499.875000,0.000000
100003,1.500000,0.0,-1400.750000,0.0,-544.500000,-1097.333333,0.000,0.0,254350.125000,0.000000,202500.00000,0.0,3.500000,-816.000000,NaN
100004,2.000000,0.0,-867.000000,0.0,-488.500000,-532.500000,0.000,0.0,94518.900000,0.000000,0.00000,0.0,3.000000,-532.000000,NaN
100005,0.666667,0.0,-190.666667,0.0,439.333333,-123.000000,0.000,0.0,219042.000000,189469.500000,0.00000,0.0,3.333333,-54.333333,1420.500000


In [71]:
bureau.dtypes

CREDIT_ACTIVE             float64
CREDIT_CURRENCY           float64
DAYS_CREDIT               float64
CREDIT_DAY_OVERDUE        float64
DAYS_CREDIT_ENDDATE       float64
DAYS_ENDDATE_FACT         float64
AMT_CREDIT_MAX_OVERDUE    float64
CNT_CREDIT_PROLONG        float64
AMT_CREDIT_SUM            float64
AMT_CREDIT_SUM_DEBT       float64
AMT_CREDIT_SUM_LIMIT      float64
AMT_CREDIT_SUM_OVERDUE    float64
CREDIT_TYPE               float64
DAYS_CREDIT_UPDATE        float64
AMT_ANNUITY               float64
dtype: object

In [72]:
X_train_merge = pd.merge(X_train_merge, bureau, left_index=True, right_index=True, how='left')
X_test_merge = pd.merge(X_test_merge, bureau, left_index=True, right_index=True, how='left')

X_train_merge.shape, X_test_merge.shape

((54615, 154), (48744, 154))

In [73]:
#lo mismo para el data_train
col_mean_train = (X_train_merge.loc[:,X_train_merge.count()<X_train_merge.shape[0]].min())
dict_col_mean_train = {}
for i in range(len(col_mean_train)):
    dict_col_mean_train[col_mean_train.index[i]] = col_mean_train[i]
    
#lo mismo para el data_test
col_mean_test = (X_test_merge.loc[:,X_test_merge.count()<X_test_merge.shape[0]].min())
dict_col_mean_test = {}
for i in range(len(col_mean_test)):
    dict_col_mean_test[col_mean_test.index[i]] = col_mean_test[i]

In [74]:
col_mean_train

CREDIT_ACTIVE                  0.00000
CREDIT_CURRENCY                0.00000
DAYS_CREDIT                -2922.00000
CREDIT_DAY_OVERDUE             0.00000
DAYS_CREDIT_ENDDATE       -21342.00000
DAYS_ENDDATE_FACT          -8376.50000
AMT_CREDIT_MAX_OVERDUE         0.00000
CNT_CREDIT_PROLONG             0.00000
AMT_CREDIT_SUM                 0.00000
AMT_CREDIT_SUM_DEBT      -161159.44500
AMT_CREDIT_SUM_LIMIT      -97891.66125
AMT_CREDIT_SUM_OVERDUE         0.00000
CREDIT_TYPE                    0.00000
DAYS_CREDIT_UPDATE        -20939.00000
AMT_ANNUITY                    0.00000
dtype: float64

In [75]:
col_mean_test

CREDIT_ACTIVE                0.0
CREDIT_CURRENCY              0.0
DAYS_CREDIT              -2921.0
CREDIT_DAY_OVERDUE           0.0
DAYS_CREDIT_ENDDATE      -2759.0
DAYS_ENDDATE_FACT        -2857.0
AMT_CREDIT_MAX_OVERDUE       0.0
CNT_CREDIT_PROLONG           0.0
AMT_CREDIT_SUM               0.0
AMT_CREDIT_SUM_DEBT          0.0
AMT_CREDIT_SUM_LIMIT         0.0
AMT_CREDIT_SUM_OVERDUE       0.0
CREDIT_TYPE                  0.0
DAYS_CREDIT_UPDATE       -2907.0
AMT_ANNUITY                  0.0
dtype: float64

In [76]:
X_train_merge = X_train_merge.fillna(value=dict_col_mean_train)
X_test_merge = X_test_merge.fillna(value=dict_col_mean_test)

In [77]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier()
GBC.fit(X_train_merge, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [78]:
GBC.score(X_train_merge, y_train)

0.70480637187585826

In [79]:
predict = GBC.predict_proba(X_test_merge)[:,1]

In [51]:
predict

array([ 0.31084479,  0.53747707,  0.20637275, ...,  0.36942974,
        0.45510901,  0.76968351])

In [52]:
#sin correlación de datos
result = pd.DataFrame(data=predict, index=X_test_merge.index, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_34.csv')

### Ahora uniré la tabla de credit card balance

In [80]:
credict_cv = pd.read_csv('./data/credit_card_balance.csv')
credict_cv.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [81]:
credict_cv = credict_cv.set_index(credict_cv.columns[1])
credcit_cv = credict_cv.drop('SK_ID_PREV',axis=1)
print(credict_cv.shape)
credict_cv.head()

(3840312, 22)


,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
378907,2562384,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
363914,2582071,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
371185,1740877,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
337855,1389973,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
126868,1891521,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [82]:
credict_cv = label_encoder(credict_cv)

In [89]:
X_train_merge.shape, X_test_merge.shape

((54615, 154), (48744, 154))

In [86]:
credict_cv = credict_cv.groupby(credict_cv.index).mean()
credict_cv.shape

(103558, 22)

In [88]:
credict_cv.filter(X_train_merge.index, axis=0).shape, credict_cv.filter(X_test_merge.index, axis=0).shape

((15788, 22), (16653, 22))

In [90]:
X_train_merge2 = pd.merge(X_train_merge, credict_cv, left_index=True, right_index=True, how='left')
X_test_merge2 = pd.merge(X_test_merge, credict_cv, left_index=True, right_index=True, how='left')

X_train_merge2.shape, X_test_merge2.shape

((54615, 176), (48744, 176))

In [91]:
#lo mismo para el data_train
col_mean_train = (X_train_merge2.loc[:,X_train_merge2.count()<X_train_merge2.shape[0]].min())
dict_col_mean_train = {}
for i in range(len(col_mean_train)):
    dict_col_mean_train[col_mean_train.index[i]] = col_mean_train[i]
    
#lo mismo para el data_test
col_mean_test = (X_test_merge2.loc[:,X_test_merge2.count()<X_test_merge2.shape[0]].min())
dict_col_mean_test = {}
for i in range(len(col_mean_test)):
    dict_col_mean_test[col_mean_test.index[i]] = col_mean_test[i]

In [92]:
X_train_merge2 = X_train_merge2.fillna(value=dict_col_mean_train)
X_test_merge2 = X_test_merge2.fillna(value=dict_col_mean_test)

In [94]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier()
GBC.fit(X_train_merge2, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [95]:
GBC.score(X_train_merge2,  y_train)

0.70834019957887029

In [97]:
predict = GBC.predict_proba(X_test_merge2)[:,1]

## Mejor score por ahora .747

In [98]:
#sin correlación de datos
result = pd.DataFrame(data=predict, index=X_test_merge.index, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_35.csv')

### Cargaré el POS_CASH_BALANCE

In [99]:
posh_cv = pd.read_csv('./data/POS_CASH_balance.csv')

In [100]:
posh_cv.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [101]:
posh_cv = posh_cv.set_index(posh_cv.columns[1])
posh_cv = posh_cv.drop('SK_ID_PREV',axis=1)
posh_cv.shape

(10001358, 6)

In [102]:
posh_cv = label_encoder(posh_cv)

In [104]:
posh_cv = posh_cv.groupby(posh_cv.index).mean()
posh_cv.shape

(337252, 6)

In [105]:
posh_cv.filter(X_train_merge2.index, axis=0).shape, posh_cv.filter(X_test_merge2.index, axis=0).shape

((51672, 6), (47808, 6))

In [106]:
X_train_merge3 = pd.merge(X_train_merge2, posh_cv, left_index=True, right_index=True, how='left')
X_test_merge3 = pd.merge(X_test_merge2, posh_cv, left_index=True, right_index=True, how='left')

X_train_merge3.shape, X_test_merge3.shape

((54615, 182), (48744, 182))

In [107]:
#lo mismo para el data_train
col_mean_train = (X_train_merge3.loc[:,X_train_merge3.count()<X_train_merge3.shape[0]].min())
dict_col_mean_train = {}
for i in range(len(col_mean_train)):
    dict_col_mean_train[col_mean_train.index[i]] = col_mean_train[i]
    
#lo mismo para el data_test
col_mean_test = (X_test_merge3.loc[:,X_test_merge3.count()<X_test_merge3.shape[0]].min())
dict_col_mean_test = {}
for i in range(len(col_mean_test)):
    dict_col_mean_test[col_mean_test.index[i]] = col_mean_test[i]

In [108]:
X_train_merge3 = X_train_merge3.fillna(value=dict_col_mean_train)
X_test_merge3 = X_test_merge3.fillna(value=dict_col_mean_test)

In [109]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier()
GBC.fit(X_train_merge3, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [110]:
GBC.score(X_train_merge3, y_train)

0.7105007781744942

In [111]:
predict = GBC.predict_proba(X_test_merge3)[:,1]

In [112]:
predict

array([ 0.35435934,  0.5209678 ,  0.23266632, ...,  0.32469817,
        0.36929862,  0.72570799])

In [113]:
#sin correlación de datos
result = pd.DataFrame(data=predict, index=X_test_merge.index, columns=['TARGET'])
result.index.name = 'SK_ID_CURR'
result.to_csv('./results/submission_test_36.csv')